<a href="https://colab.research.google.com/github/MikeCorv/llm_evals/blob/main/MMLU_TEST_QWEN2_5B_LLAMA3B_1_5SHOTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q lm_eval[vllm,gpu] accelerate


In [ ]:
pip install -U bitsandbytes

In [ ]:
import lm_eval
import torch
import bitsandbytes
import json
import gc
from google.colab import userdata, drive
from huggingface_hub import login
import os

In [ ]:
#HF KEYS
login(userdata.get('HF_TOKEN'))

In [ ]:
# 1. Mount Drive
drive.mount('/content/drive')

# 2. Create a folder for your results
save_folder = "/content/drive/MyDrive/LLM_Results"

print(f"📂 Results will be saved to: {save_folder}")

In [ ]:
task_name = "mmlu"

In [ ]:
# Clear memory
torch.cuda.empty_cache()
gc.collect()

In [ ]:
shots_list = [x for x in range (6)]
print(shots_list)

In [ ]:
model_id_1 = "Qwen/Qwen2.5-1.5B-Instruct"
qwen_scores = []

In [ ]:
results_test = lm_eval.simple_evaluate(
      model="hf",
      model_args=f"pretrained={model_id_1},load_in_4bit=True,trust_remote_code=True",
      tasks=[task_name],
      num_fewshot=2,
      limit = 10,
      verbosity = "WARNING",
      apply_chat_template=True,
      fewshot_as_multiturn=True,
      log_samples = True,
      batch_size = 8
  )

In [ ]:
for key in results_test.keys():
  print(key)

In [ ]:
print(json.dumps(results_test['samples'], indent=4))

In [ ]:
for shot_num in shots_list:
  results = lm_eval.simple_evaluate(
      model="hf",
      model_args=f"pretrained={model_id_1},load_in_4bit=True,trust_remote_code=True",
      tasks=[task_name],
      num_fewshot=shot_num,
      limit = 10,
      verbosity = "WARNING"
  )


  entry = {
            "model_id": model_id_1,
            "shots": shot_num,
            "results": json.dumps(results['results'], indent = 2) # This contains the total MMLU + all subtasks
        }
  qwen_scores.append(entry)



In [ ]:
save_file = f"{save_folder}/qwen_scores_0_to_5.json"

# Save the list to JSON
with open(save_file, "w") as f:
    json.dump(qwen_scores, f, indent=4)

print(f"💾 Success! Data saved to: {save_file}")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import whoami
try:
    user_info = whoami()
    print(f"✅ Success! Logged in as: {user_info['name']}")
    print(f"✅ Token permissions: {user_info['auth']['accessToken']['role']}")
except Exception as e:
    print("❌ Still failing. Error:", e)

In [ ]:
model_id_2 = "meta-llama/Llama-3.2-3B-Instruct"
llama_scores = []

In [ ]:
for shot_num in shots_list:
  results = lm_eval.simple_evaluate(
      model="hf",
      model_args=f"pretrained={model_id_2},load_in_4bit=True,trust_remote_code=True",
      tasks=[task_name],
      num_fewshot=shot_num,
      limit = 10,
      verbosity = "WARNING",
      batch_size = "auto"

  )
  entry = {
            "model_id": model_id_2,
            "shots": shot_num,
            "results": json.dumps(results['results'], indent = 2) # This contains the total MMLU + all subtasks
        }
  llama_scores.append(entry)

In [ ]:
print(llama_scores)

In [ ]:
save_file_2 = f"{save_folder}/llama_scores_0_to_5.json"

# Save the list to JSON
with open(save_file_2, "w") as f:
    json.dump(llama_scores, f, indent=4)

print(f"💾 Success! Data saved to: {save_file_2}")

In [ ]:
results_llama_json = json.dumps(results_Llama32_1B['results'], indent = 2)
print(results_llama_json)